In [ ]:
import math
from autumn.core.project import get_project
from matplotlib import pyplot
from autumn.core.plots.utils import REF_DATE
# from autumn.calibration.targets import get_target_series
import pandas as pd
from autumn.core.utils.display import pretty_print

from autumn.core.plots.uncertainty.plots import _get_target_values, _plot_targets_to_axis

In [ ]:
project = get_project("sm_sir", "ho_chi_minh_city")

In [ ]:
update_params = {
#       "contact_rate": 0.115,
#       "prop_icu_among_hospitalised": 0.15,
#     "immunity_stratification.infection_risk_reduction.low": 0.25,
#     "immunity_stratification.infection_risk_reduction.high": 0.4,
#     "mobility.microdistancing.behavior.parameters.max_effect": 0.2,
#     "age_stratification.prop_hospital.multiplier": 1.5,
#     "infectious_seed": 1447.6389411122898,
#     "sojourns.active.proportion_early": 0.7,
#     "sojourns.active.total_time": 8.,
#     "sojourns.latent.proportion_early": 0.7,
#     "sojourns.latent.total_time": 5.,
#     "voc_emergence.omicron.contact_rate_multiplier": 1.5,
#     "voc_emergence.omicron.icu_multiplier": 0.4,
#     "voc_emergence.omicron.relative_active_period": 1.0,
#     "voc_emergence.omicron.relative_latency": 0.67,
#     "voc_emergence.omicron.new_voc_seed.start_time": 746.,
#     "voc_emergence.omicron.hosp_protection": 0.8, 
#     "voc_emergence.omicron.death_protection": 0.9,
#     "voc_emergence.wild_type.relative_active_period": 1.912747083227452
#     "voc_emergence.wild_type.relative_latency": 1.23511172131566
}
params = project.param_set.baseline.update(update_params, calibration_format=True)


In [ ]:
pretty_print(params)

In [ ]:
model = project.run_baseline_model(params)
derived_df = model.get_derived_outputs_df()

In [ ]:
derived_df.columns

In [ ]:
outputs = [
#     "notifications", 
    "hospital_occupancy", 
#     "infection_deaths",
    "icu_occupancy",
    "prop_ever_infected",
    "hospital_admissionsXagegroup_60",
    "hospital_occupancyXagegroup_60"
] #, "hospital_admissions", "hospital_occupancy", "icu_admissions", "icu_occupancy"]
for output in outputs:
    fig = pyplot.figure(figsize=(12, 8))
    pyplot.style.use("ggplot")
    axis = fig.add_subplot()

    axis = derived_df[output].plot()
    axis.set_title(output)
    
    if output in project.plots:
        values, times = _get_target_values(project.plots, output)        
        date_times = pd.to_datetime(times, origin="01Jan2020",unit="D") 
        _plot_targets_to_axis(axis, values, date_times, on_uncertainty_plot=True)



In [ ]:
derived_df["icu_occupancy"] - ( derived_df["icu_occupancyXagegroup_0"] \
                              + derived_df["icu_occupancyXagegroup_15"] \
                              + derived_df["icu_occupancyXagegroup_25"] \
                              + derived_df["icu_occupancyXagegroup_50"] \
                              + derived_df["icu_occupancyXagegroup_60"])

In [ ]:
# Check that the functions that return hospital and ICU admissions and occupancy by age group 
# are equal to the total values
# print false if there is no values in the different that is greater than 1e-10
print((derived_df['hospital_admissions'] - derived_df.loc[:, ['hospital_admissionsXagegroup_0',
                   'hospital_admissionsXagegroup_15',
                   'hospital_admissionsXagegroup_25',
                   'hospital_admissionsXagegroup_50',
                   'hospital_admissionsXagegroup_60']].sum(axis=1) > 1e-10).any() == False)

print((derived_df['hospital_occupancy'] - derived_df.loc[:, ['hospital_occupancyXagegroup_0',
                   'hospital_occupancyXagegroup_15',
                   'hospital_occupancyXagegroup_25',
                   'hospital_occupancyXagegroup_50',
                   'hospital_occupancyXagegroup_60']].sum(axis=1) > 1e-10).any() == False)

print((derived_df['icu_admissions'] - derived_df.loc[:, ['icu_admissionsXagegroup_0',
                   'icu_admissionsXagegroup_15',
                   'icu_admissionsXagegroup_25',
                   'icu_admissionsXagegroup_50',
                   'icu_admissionsXagegroup_60']].sum(axis=1) > 1e-10).any() == False)

print((derived_df['icu_occupancy'] - derived_df.loc[:, ['icu_occupancyXagegroup_0',
                   'icu_occupancyXagegroup_15',
                   'icu_occupancyXagegroup_25',
                   'icu_occupancyXagegroup_50',
                   'icu_occupancyXagegroup_60']].sum(axis=1) > 1e-10).any() == False)


In [ ]:
derived_df